In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-18 15:04:31--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.001s  

2025-10-18 15:04:31 (65.2 MB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [3]:
df = pd.read_csv('course_lead_scoring.csv')

In [4]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [8]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [9]:
df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

In [10]:
df.industry.mode()

0    retail
Name: industry, dtype: object

In [11]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [12]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [14]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [15]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [17]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [18]:
df_full_train = df_full_train.reset_index(drop=True)

In [19]:
def mutual_info_column_score(series):
    return mutual_info_score(df_full_train.converted, series)

In [20]:
mi = round(df_full_train[categorical].apply(mutual_info_column_score), 2)
mi.sort_values(ascending=False)

lead_source          0.03
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

In [21]:
df_full_train[numerical].corrwith(df_full_train.converted)

number_of_courses_viewed    0.442068
annual_income               0.029612
interaction_count           0.378482
lead_score                  0.225641
dtype: float64

In [36]:
train_dicts = df_train[categorical].to_dict(orient='records')

In [37]:
dv = DictVectorizer(sparse=False)

In [38]:
dv.fit(train_dicts)
X_train = dv.transform(train_dicts)

In [39]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [40]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [41]:
y_pred = model.predict_proba(X_val)[:, 1]

In [42]:
converted_descision = (y_pred >= 0.5)

In [43]:
accuracy = y_val == converted_descision

In [56]:
original_a = accuracy.mean()     

In [52]:
train_dictsone = df_train[['industry', 'employment_status', 'location']].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dictsone)
X_trainone = dv.transform(train_dictsone)
val_dictsone = df_val[['industry', 'employment_status', 'location']].to_dict(orient='records')
X_valone = dv.transform(val_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_trainone, y_train)
y_pred = model.predict_proba(X_valone)[:, 1]
converted_descision = (y_pred >= 0.5)
accuracy = y_val == converted_descision
lead_score = accuracy.mean()

In [53]:
train_dictstwo = df_train[['lead_source', 'employment_status', 'location']].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dictstwo)
X_traintwo = dv.transform(train_dictstwo)
val_dictstwo = df_val[['lead_source', 'employment_status', 'location']].to_dict(orient='records')
X_valtwo = dv.transform(val_dictstwo)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_traintwo, y_train)
y_pred = model.predict_proba(X_valtwo)[:, 1]
converted_descision = (y_pred >= 0.5)
accuracy = y_val == converted_descision
industry = accuracy.mean()

In [54]:
train_dictsthree = df_train[['lead_source', 'industry', 'location']].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dictsthree)
X_trainthree = dv.transform(train_dictsthree)
val_dictsthree = df_val[['lead_source', 'industry', 'location']].to_dict(orient='records')
X_valthree = dv.transform(val_dictsthree)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_trainthree, y_train)
y_pred = model.predict_proba(X_valthree)[:, 1]
converted_descision = (y_pred >= 0.5)
accuracy = y_val == converted_descision
employment_status = accuracy.mean()

In [59]:
print(original_a - lead_score)
print(original_a - industry)
print(original_a - employment_status)

-0.07167235494880542
-0.058020477815699634
0.0


In [69]:
train_dictsfour = df_train[categorical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dictsthree)
X_trainfour = dv.transform(train_dictsfour)
val_dictsfour = df_val[categorical].to_dict(orient='records')
X_valfour = dv.transform(val_dictsfour)
model = LogisticRegression(solver='liblinear', C=100, max_iter=1000, random_state=42)
model.fit(X_trainfour, y_train)
y_pred = model.predict_proba(X_valfour)[:, 1]
converted_descision = (y_pred >= 0.5)
accuracy = y_val == converted_descision
c_acuracy = accuracy.mean()

In [70]:
c_acuracy

np.float64(0.5494880546075085)

In [ ]:
1 = 0.5460750853242321
0.1 = 0.5631399317406144
0.01 = 0.5563139931740614
10 = 0.5460750853242321
100 = 0.5494880546075085